# Stock Market Analysis for Swing Trading
Shifa Salsabiila

## Table of Content
[1. Background](#1.-Background)<br>
[2. Goals](#2.-Goals)<br>
[3. Historical Aggregates](3.-#Historical-Aggregates)<br>
&nbsp;&nbsp;&nbsp; [3.1. API Call](#3.1.0API-Call)<br>
&nbsp;&nbsp;&nbsp; [3.2. Data Preprocessing](#Data-Preprocessing)<br>
&nbsp;&nbsp;&nbsp; [3.3. Visualization](#Visualization)<br>
[4. ]()

## 1. Background
> This notebook is made as a personal initial analysis on swing trading strategies, potential returns, and risks. The analysis performed will solely focus on the stock prices itself without any consideration of external factors and may involve personal opinions.

Attempting to make money off the stock market takes on many forms. Some people choose to pursue the route of long term investing on companies they believe will succeed in the future, while some others choose to capitalize on the short term fluctuation of stock prices. Long term investments usually aims to make larger gains in the long run, aiming for returns of possibly more than 30% over a span of perhaps 5, 10, or even 15 years. Long term investing is built on top of a company's value analysis, this means that a person would invest on a company they believe have good value that will continue to improve over the years. 

Now let's talk about trading. Trading is stretched over a shorter time period than investing, sometimes over a period of a day (day traders) or a little longer of upto weeks (swing traders). In trading, the value or expected value of a company isn't really a key aspect to observe. Instead, daily and weekly price trends become the key metrics to help traders decide what stocks they want to buy, when to buy them, and when to sell them. The gains of a single trade is usually lower than that of investing, but since it takes place over a much shorter period of time, the frequency of achieving the actual gains is usually higher. 

What makes trading, specifically swing trading intersting to me is that intuitively, small gains of around 4 - 10% happens a lot more often than larger gains. So instead of letting the price of your assets fluctuate up and down over a long period of time until eventually you reach a high gain value, you can take smaller and faster gains, minimize the price drops and build a compounding return from those earlier gains. 

## 2. Goals
This analysis focuses on swing trading, with the attempt to answer the following questions: <br>
&nbsp;&nbsp;&nbsp; 1.  <br>
&nbsp;&nbsp;&nbsp; 2.  <br>

## Depdendencies

In [610]:
import pandas as pd
import requests
import os
import scipy

from enum import Enum
from dotenv import load_dotenv
from datetime import date, datetime
from typing import Optional, List
import plotly.graph_objects as go
import plotly.figure_factory as ff

In [611]:
load_dotenv()

API_KEY = os.getenv('API_KEY')

## Historical Aggregates

Endpoint: /v2/aggs/ticker/{stocksTicker}/range/{multiplier}/{timespan}/{from}/{to}

[Documentation Link](https://polygon.io/docs/stocks/get_v2_aggs_ticker__stocksticker__range__multiplier___timespan___from___to)

### API Call

In [612]:
# Path and parameter configurations
stocks_ticker = 'TSLA'
multiplier = '1'
timespan = 'day'
from_date = '2021-01-01'
to_date = date.today().strftime('%Y-%m-%d')

URL = f'https://api.polygon.io/v2/aggs/ticker/{stocks_ticker}/range/{multiplier}/{timespan}/{from_date}/{to_date}?adjusted=true&sort=asc&apiKey={API_KEY}'


In [613]:
r = requests.get(url = URL)
data = r.json()

### Data Preprocessing

Notes:
* For simplicity, the definition of daily gain/loss here is defined to be close - open price.

In [614]:
df = pd.json_normalize(data['results']) 

# Rename columns
df.rename(columns = {
    'v': 'volume',
    'vw': 'avg',
    'o': 'open',
    'c': 'close',
    'h': 'high',
    'l': 'low',
    't': 'window_start_timestamp',
    'n': 'number_of_trx'
}, inplace=True)

# Add date column
df['date'] = df['window_start_timestamp'].apply(lambda x: (datetime.fromtimestamp(x/1000).date()))

# Add gain_loss
df = df.assign(gain_loss = lambda x: ((x['close'] - x['open'])/100))

# Explicitly add index as row column
df['row_index'] = df.index

# Add time to win column
df['time_to_win'] = -1

df

,volume,avg,open,close,high,low,window_start_timestamp,number_of_trx,date,gain_loss,row_index,time_to_win
0,88310985.0,237.0286,239.3200,234.9133,241.3333,234.4500,1619496000000,907369,2021-04-27,-0.044067,0,-1
1,66663141.0,233.0145,232.1367,231.4667,236.1667,231.2000,1619582400000,694144,2021-04-28,-0.006700,1,-1
2,86536347.0,227.0697,233.1700,225.6667,234.0833,222.8334,1619668800000,899789,2021-04-29,-0.075033,2,-1
3,122276166.0,232.9178,222.5300,236.4800,238.4900,222.0467,1619755200000,1014027,2021-04-30,0.139500,3,-1
4,81129429.0,230.0541,234.6000,228.3000,235.3333,226.8333,1620014400000,761158,2021-05-03,-0.063000,4,-1
...,...,...,...,...,...,...,...,...,...,...,...,...
499,210925047.0,164.9342,166.1650,162.9900,169.7000,160.5600,1681963200000,2473944,2023-04-20,-0.031750,499,-1
500,123538954.0,164.1298,164.8000,165.0800,166.0000,161.3208,1682049600000,1209577,2023-04-21,0.002800,500,-1
501,140006559.0,161.4701,164.6500,162.5500,165.6500,158.6100,1682308800000,1306873,2023-04-24,-0.021000,501,-1
502,121999312.0,161.7449,159.8200,160.6700,163.4700,158.7500,1682395200000,1054894,2023-04-25,0.008500,502,-1


### Visualization

In [615]:
fig = go.Figure(data=[go.Candlestick(x=df['date'],
                open=df['open'],
                high=df['high'],
                low=df['low'],
                close=df['close'])])

fig.show()

## 4. Descriptive Analytics

### 4.1. Overall Statistical Description

In [616]:
df.describe()

,volume,avg,open,close,high,low,window_start_timestamp,number_of_trx,gain_loss,row_index,time_to_win
count,5.040000e+02,504.000000,504.000000,504.000000,504.000000,504.000000,5.040000e+02,5.040000e+02,504.000000,504.000000,504.0
mean,9.407242e+07,250.171837,250.441056,250.110387,256.153527,244.164201,1.650871e+12,8.960607e+05,-0.003307,251.500000,-1.0
std,4.413812e+07,64.200859,64.398571,64.193888,65.894933,62.544449,1.825612e+10,4.214754e+05,0.087112,145.636534,0.0
min,2.940167e+07,109.024800,103.000000,108.100000,111.750000,101.810000,1.619496e+12,2.564000e+05,-0.500333,0.000000,-1.0
25%,6.391618e+07,202.356450,202.383775,202.127525,207.572525,198.529975,1.635070e+12,6.059850e+05,-0.046834,125.750000,-1.0
50%,8.235190e+07,239.443450,239.380000,238.600000,244.766700,234.558350,1.650902e+12,8.038415e+05,-0.000008,251.500000,-1.0
75%,1.086685e+08,296.786025,298.625000,296.761700,303.934150,289.347500,1.666606e+12,1.110951e+06,0.045400,377.250000,-1.0
max,3.065420e+08,411.149700,411.470000,409.970000,414.496700,405.666700,1.682482e+12,2.631365e+06,0.334600,503.000000,-1.0


### 4.2. Daily Gain/Loss Distribution

**Summary**<br>
As it can be seen, daily gains and losses seem to follow a balanced normal curve with a skew value relatively close to zero. 

In [617]:
df['gain_loss'].describe()

count    504.000000
mean      -0.003307
std        0.087112
min       -0.500333
25%       -0.046834
50%       -0.000008
75%        0.045400
max        0.334600
Name: gain_loss, dtype: float64

In [618]:
fig = ff.create_distplot([df['gain_loss'].to_list()], ['gain_loss'], bin_size=.01)
fig.show()

### 4.2. Weekly Gain/Loss

### 4.3. Average Time to Reach 0.05 (Max Days)

Using a window size of 1 day, counting from each day, how many days does it take to reach a gain of 0.05, with a maximum hold day of 7. Count as fail if the maximum hold day is reached. 

In [619]:
# Enums
class ComparisonType(str, Enum):
    AVG = "avg"
    OPEN = "open"
    CLOSE = "close"
    HIGH = "high"
    LOW = "low"

In [620]:
# Supporting Functions
def get_value(
        df: pd.DataFrame, 
        idx_start: int, 
        idx_end: int,
        comparator: ComparisonType
    ) -> float:
    """
    Returns the gain/loss value between two dates
    [Input]
        df: source dataframe that will be analysed
        idx_start: starting date row
        idx_end: ending date row
        comparator: what metric to use as comparison
    [Output]
        float
    """
    start_val = df[comparator.value].iloc[idx_start]
    end_val = df[comparator.value].iloc[idx_end]
    
    return (end_val - start_val)/abs(start_val)

def window_slide(
        row: pd.Series,
        df: pd.DataFrame, 
        x: float,
        idx_start: int, 
        max_days: int,
        comparator: ComparisonType
    ) -> (int, float):
    """
    Returns a tuple that denotes the minimum number of days it takes to reach a certain gain value from 
    a starting date along with its gain value. Returns -1 if fail condition is reached before win. The fail
    condition is in the form of a maximum number of days. 
    [Input]
        row: the row from the dataframe that denotes the starting date
        df: source dataframe that will be analysed
        x: target gain to be reached
        idx_start: the index of the starting row
        max_days: maximum days of withholding before declaring failure
        comparator: what metric to use as comparison
    [Output]
        (int, float)
    """
    idx_curr = idx_start
    gain_curr = 0
    while (gain_curr < x) and (idx_curr - idx_start < max_days) and (idx_curr < len(df)-1):
        idx_curr += 1
        gain_curr = get_value(df=df, idx_start=idx_start, idx_end=idx_curr, comparator=comparator)

    # Success condition reached
    if (gain_curr > x):
        return idx_curr - idx_start, gain_curr
    
    # Fail condition reached
    return -1, gain_curr


In [621]:
def time_to_reach_x(
        df: pd.DataFrame,
        x: float = 0.05, 
        max_days: int = 14,
        comparison_type: ComparisonType = ComparisonType.AVG
    ) -> pd.DataFrame:
    """
    For each day that the market is open, this function calculates how many days it takes to reach a gain of x,
    with an optional fail condition of either maximum loss threshold or maximum numer of withholding days.
    [Input]
        x: float
        loss_threshold: float
        max_days: int
    [Output]
        pd.DataFrame
    """
    # Initiate new dataframes
    res = pd.DataFrame()
    temp = pd.DataFrame()
    
    temp['window_res'] = df.apply(lambda row : window_slide(row, df, x, row['row_index'], max_days, comparison_type), axis = 1)
    res['start_date'] = df['date']
    res['time_to_win'], res['gain'] = zip(*temp['window_res'])

    return res
    

In [622]:
result = time_to_reach_x(df, x=0.05, max_days=40)
result

,start_date,time_to_win,gain
0,2021-04-27,-1,-0.085316
1,2021-04-28,-1,-0.020836
2,2021-04-29,-1,-0.003557
3,2021-04-30,-1,-0.017512
4,2021-05-03,-1,-0.013768
...,...,...,...
499,2023-04-20,-1,-0.055227
500,2023-04-21,-1,-0.050597
501,2023-04-24,-1,-0.034959
502,2023-04-25,-1,-0.036598


In [623]:
wins = result.drop(result[result['time_to_win'] == -1].index)
wins

,start_date,time_to_win,gain
9,2021-05-10,32,0.065880
10,2021-05-11,30,0.063221
11,2021-05-12,29,0.081221
12,2021-05-13,9,0.064160
13,2021-05-14,8,0.057394
...,...,...,...
478,2023-03-21,8,0.051526
481,2023-03-24,5,0.074510
482,2023-03-27,4,0.051452
483,2023-03-28,3,0.081903


In [624]:
win_rate = len(wins)/len(result)
win_rate

0.6964285714285714

In [625]:
result.describe()

,time_to_win,gain
count,504.000000,504.000000
mean,6.867063,-0.016922
std,9.775623,0.154734
min,-1.000000,-0.506025
25%,-1.000000,-0.081930
50%,3.000000,0.057059
75%,10.000000,0.075686
max,40.000000,0.148889


In [626]:
fig = ff.create_distplot([wins['time_to_win'].to_list()], ['time_to_win'])
fig.show()

### 4.4. Average Time to Reach 0.05 (Loss Threshold)

TODO: Using a window size of 1 day, counting from each day, how many days does it take to reach a gain of 0.05, with a maximum loss of also 0.05. Count as fail if the loss threshold is met.